In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
df = pd.read_csv('data/data_products.csv', index_col=None)
df.shape

(371536, 10)

In [3]:
df['agrupacion_unica'] = df.product_id.astype(str) + df.category_id.astype(str) + df.brand + df.price.astype(str) + df.gmv.astype(str)
items = [i for i in range(1,df.agrupacion_unica.nunique()+1)]
df['item'] = items # se asume que los productos con precios, categorias y vistas diferentes son items diferentes, puede ser por vendendor, etc etc ..
df.head()

,product_id,category_id,brand,price,cant_views,cant_clicks,cant_purchase,ctr,cvr,gmv,agrupacion_unica,item
0,44600062,2103807459595387724,shiseido,35.79,143,0.0,1.0,0.000000,0.006993,0.000000,446000622103807459595387724shiseido35.790.0,1
1,3900821,2053013552326770905,aqua,33.20,3019,23.0,27.0,0.007618,0.008943,0.002262,39008212053013552326770905aqua33.20.0022620565...,2
2,17200506,2053013559792632471,sin_dato,543.10,1181,0.0,5.0,0.000000,0.004234,0.000000,172005062053013559792632471sin_dato543.10.0,3
3,1307067,2053013558920217191,lenovo,251.74,17571,6.0,651.0,0.000341,0.037050,0.003185,13070672053013558920217191lenovo251.740.003184...,4
4,1004237,2053013555631882655,apple,1081.98,52856,3090.0,2249.0,0.058461,0.042550,2.691402,10042372053013555631882655apple1081.982.691402...,5


In [96]:
df.category_id.nunique()

624

In [97]:
df.item.nunique()

371536

In [98]:
df.product_id.nunique()

166794

In [4]:
df_cant_items = df[['brand','item','category_id']].groupby(['brand','category_id']).nunique().reset_index()
df_cant_items = df_cant_items.rename(columns={'item':'cant_items_brand'})
df_cant_items.head()

,brand,category_id,cant_items_brand
0,a-case,2053013558525952589,65
1,a-derma,2053013558802776671,4
2,a-derma,2053013558861496931,6
3,a-derma,2053013558895051365,13
4,a-elita,2055156924407612189,2


In [5]:
df = df.merge(df_cant_items,on=['brand','category_id'],how='left')
df.head()

,product_id,category_id,brand,price,cant_views,cant_clicks,cant_purchase,ctr,cvr,gmv,agrupacion_unica,item,cant_items_brand
0,44600062,2103807459595387724,shiseido,35.79,143,0.0,1.0,0.000000,0.006993,0.000000,446000622103807459595387724shiseido35.790.0,1,4
1,3900821,2053013552326770905,aqua,33.20,3019,23.0,27.0,0.007618,0.008943,0.002262,39008212053013552326770905aqua33.20.0022620565...,2,14
2,17200506,2053013559792632471,sin_dato,543.10,1181,0.0,5.0,0.000000,0.004234,0.000000,172005062053013559792632471sin_dato543.10.0,3,387
3,1307067,2053013558920217191,lenovo,251.74,17571,6.0,651.0,0.000341,0.037050,0.003185,13070672053013558920217191lenovo251.740.003184...,4,732
4,1004237,2053013555631882655,apple,1081.98,52856,3090.0,2249.0,0.058461,0.042550,2.691402,10042372053013555631882655apple1081.982.691402...,5,3681


In [ ]:
df = df.merge(df_cant_items,on=['brand','category_id'],how='left')
df.head()

In [6]:
df.shape

(371536, 13)

In [81]:
data0 = df.groupby('category_id', group_keys=False).apply(lambda x: x.sample(frac=0.1))

In [82]:
data0.item.nunique()

37154

In [83]:
df_cant_items = data0[['item','category_id']].groupby(['category_id']).nunique().reset_index()
df_cant_items = df_cant_items.rename(columns={'item':'cant_items_cat'})
df_cant_items.head()

,category_id,cant_items_cat
0,2053013552226107603,7
1,2053013552259662037,627
2,2053013552293216471,162
3,2053013552326770905,238
4,2053013552351936731,184


In [84]:
df_gmv = data0[['gmv','category_id']].groupby(['category_id']).mean().reset_index()
df_gmv = df_gmv.rename(columns={'gmv':'median_gmv'})
df_gmv = df_gmv.merge(df_cant_items,on=['category_id'],how='left')
df_gmv.head()

,category_id,median_gmv,cant_items_cat
0,2053013552226107603,0.005592,7
1,2053013552259662037,0.051220,627
2,2053013552293216471,0.007049,162
3,2053013552326770905,0.030467,238
4,2053013552351936731,0.019565,184


In [100]:
(df_gmv[df_gmv.median_gmv > 0].category_id.nunique())/df_gmv.category_id.nunique()

0.37755102040816324

In [85]:
data0 = data0.merge(df_gmv,on=['category_id'],how='left')
data0.head()

,product_id,category_id,brand,price,cant_views,cant_clicks,cant_purchase,ctr,cvr,gmv,agrupacion_unica,item,cant_items_brand,median_gmv,cant_items_cat
0,8600147,2053013552226107603,polaris,38.53,49,0.0,0.0,0.000000,0.000000,0.000000,86001472053013552226107603polaris38.530.0,315113,19,0.005592,7
1,8600227,2053013552226107603,sin_dato,13.77,11,0.0,0.0,0.000000,0.000000,0.000000,86002272053013552226107603sin_dato13.770.0,37664,19,0.005592,7
2,8600037,2053013552226107603,elenberg,20.57,57,1.0,2.0,0.017544,0.035088,0.012662,86000372053013552226107603elenberg20.570.01266...,147728,13,0.005592,7
3,8600043,2053013552226107603,elenberg,14.13,62,1.0,1.0,0.016129,0.016129,0.003676,86000432053013552226107603elenberg14.130.00367...,79351,13,0.005592,7
4,8600067,2053013552226107603,elenberg,17.99,183,2.0,9.0,0.010929,0.049180,0.009669,86000672053013552226107603elenberg17.990.00966...,12714,13,0.005592,7


In [86]:
data0.isnull().sum()

product_id          0
category_id         0
brand               0
price               0
cant_views          0
cant_clicks         0
cant_purchase       0
ctr                 0
cvr                 0
gmv                 0
agrupacion_unica    0
item                0
cant_items_brand    0
median_gmv          0
cant_items_cat      0
dtype: int64

In [87]:
data0 = data0[(data0.median_gmv>0)&(data0.cant_items_cat<=500)]
data0.shape

(19453, 15)

In [88]:
data0.category_id.nunique()

216

In [89]:
data0.item.nunique()

19453

In [ ]:
def scale_list_inverse(values):
    n = len(values)
    scaled_values = [v * (n-i) for i, v in enumerate(values)]
    return scaled_values

In [90]:
def calculate_gmv(gmv_items,items_org, items_new):
    n = len(items_new)
    gmv_scaled = []
    for i,v in zip(items_org, gmv_items): 
        index = int(np.where(items_new == i)[0])
        scaled_values = v * (n-index)
        gmv_scaled.append(scaled_values)
    return(sum(gmv_scaled))

In [91]:
def two_opt_local_search(items_cat, gmv_items):
    n = len(items_cat)
    best_items = items_cat 
    random.shuffle(best_items)
    first_best_items = best_items
    best_gmv = calculate_gmv(gmv_items,items_cat,best_items)
    first_best_gmv = best_gmv
    #print(first_best_items[0:10], 'gmv: ', first_best_gmv)
    improved = True
    while improved:
        improved = False
        for i in range(1, n-2):
            for j in range(i+1, n):
                if j-i == 1: continue # changes nothing, skip then
                new_items = best_items[:].copy()
                new_items[i:j] = best_items[j-1:i-1:-1] # this is the 2woptSwap

                #new_items = best_items[:i] + best_items[i:j+1][::-1] + best_items[j+1:]
                new_gmv = calculate_gmv(gmv_items,best_items,new_items)
                #print(new_items[0:10],new_gmv)
                if new_gmv > best_gmv:
                    best_items = new_items
                    best_gmv = new_gmv
                    improved = True

           
    return (best_items,best_gmv,first_best_gmv, first_best_items)

In [92]:

#cats = [2053013558920217191,2053013555631882655]
results = {}
for i in data0.category_id.unique():
    data = data0[data0.category_id == i]#.sample(frac=0.1)
    data = data.reset_index(drop=True)
    gmv_items = data.gmv.to_numpy()
    items_cat = data.item.to_numpy() 

    rank_2opt = two_opt_local_search(items_cat, gmv_items)
    gmv_rank_2opt = rank_2opt[1]
    gmv_rank_random = rank_2opt[2]
    
    data_br = data.sort_values(by=['cant_items_brand'], ascending=False)
    data_br = data_br.reset_index(drop=True)
    gmv_items_br = data_br.gmv.to_numpy()
    items_cat_br = data_br.item.to_numpy() 
    gmv_br = calculate_gmv(gmv_items_br,items_cat_br,items_cat_br)

    results[i]={
        #'category_id': i,
        'cant_items': len(items_cat),
        'gmv_order_cant_items_brand': gmv_br,
        'gmv_rank_2opt': gmv_rank_2opt,
        'gmv_rank_random': gmv_rank_random
        }
    


In [93]:
results

{2053013552226107603: {'cant_items': 7,
  'gmv_order_cant_items_brand': 0.09415314037927541,
  'gmv_rank_2opt': 0.1401826321324962,
  'gmv_rank_random': 0.12016080020646241},
 2053013552293216471: {'cant_items': 162,
  'gmv_order_cant_items_brand': 79.37284326088229,
  'gmv_rank_2opt': 122.00154336268757,
  'gmv_rank_random': 85.47994585618262},
 2053013552326770905: {'cant_items': 238,
  'gmv_order_cant_items_brand': 926.6216929510708,
  'gmv_rank_2opt': 982.3579222213144,
  'gmv_rank_random': 857.6481318127277},
 2053013552351936731: {'cant_items': 184,
  'gmv_order_cant_items_brand': 267.99084035662844,
  'gmv_rank_2opt': 510.4479926200057,
  'gmv_rank_random': 345.51778753306553},
 2053013552385491165: {'cant_items': 64,
  'gmv_order_cant_items_brand': 6.759003292569627,
  'gmv_rank_2opt': 13.142949357510092,
  'gmv_rank_random': 11.524746676046087},
 2053013552427434207: {'cant_items': 7,
  'gmv_order_cant_items_brand': 2.1244604234199147,
  'gmv_rank_2opt': 1.6015049784556137,
  

In [94]:
df_results = pd.DataFrame.from_dict(results, orient='index').reset_index().rename(columns={'index':'category_id'})
df_results

,category_id,cant_items,gmv_order_cant_items_brand,gmv_rank_2opt,gmv_rank_random
0,2053013552226107603,7,0.094153,0.140183,0.120161
1,2053013552293216471,162,79.372843,122.001543,85.479946
2,2053013552326770905,238,926.621693,982.357922,857.648132
3,2053013552351936731,184,267.990840,510.447993,345.517788
4,2053013552385491165,64,6.759003,13.142949,11.524747
...,...,...,...,...,...
211,2152289064542273825,3,0.007835,0.003918,0.003918
212,2158174724725670855,3,0.020942,0.041884,0.041884
213,2160098698049093979,3,0.048872,0.048872,0.048872
214,2165087460176953468,7,0.002878,0.002467,0.002055


In [95]:
df_results.to_csv('data/results_500.csv')

In [102]:
100*(df_results[df_results.gmv_rank_2opt > df_results.gmv_order_cant_items_brand].category_id.nunique() / df_results.category_id.nunique())

76.38888888888889

In [103]:
100*(df_results[df_results.gmv_rank_2opt > df_results.gmv_rank_random].category_id.nunique() / df_results.category_id.nunique())

90.74074074074075